<a href="https://colab.research.google.com/github/Mv0sKff/MyDeepLearning/blob/main/week_5/Assignment_PyTorch_Model_Zoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assigment: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [1]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
import torch.optim as optim
import torch.nn as nn

In [2]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:06<00:00, 24.4MB/s]


## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [3]:
model = resnet18(weights=None)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for images, labels in trainloader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")

torch.save(model.state_dict(), "resnet18_bad.pth")

device: cuda
Epoch 1/10, Loss: 1.9937786783266067
Epoch 2/10, Loss: 1.6295081381869316
Epoch 3/10, Loss: 1.4073746705520154
Epoch 4/10, Loss: 1.2745750743865967
Epoch 5/10, Loss: 1.1680835199981927
Epoch 6/10, Loss: 1.0921173945945502
Epoch 7/10, Loss: 1.0207718646943569
Epoch 8/10, Loss: 0.9603109105977043
Epoch 9/10, Loss: 0.9045927220887691
Epoch 10/10, Loss: 0.858605058728829


In [4]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 66.39%


## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs

In [5]:
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for images, labels in trainloader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}")

torch.save(model.state_dict(), "resnet18_finetuned.pth")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 124MB/s]


device: cuda
Epoch 1/10, Loss: 2.4234615723657607
Epoch 2/10, Loss: 2.1213604178619385
Epoch 3/10, Loss: 1.8419310978555679
Epoch 4/10, Loss: 1.651758260538578
Epoch 5/10, Loss: 1.470849700293541
Epoch 6/10, Loss: 1.3476452162575723
Epoch 7/10, Loss: 1.2264838124167918
Epoch 8/10, Loss: 1.138655245270729
Epoch 9/10, Loss: 1.1264814482063055
Epoch 10/10, Loss: 1.0562952621120214


In [6]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 62.20%
